In [1]:
import pandas as pd

In [2]:
url = "https://gist.githubusercontent.com/jsz4n/b7ca11015784086788022a539935d0cf/raw/a8c3abf0a31f5c0df5e0ddd76fb9b289bac9bed1/iris.csv"

In [3]:
df = pd.read_csv(url)

SVM multiclass
Essayez une approche multiclasses du svm sur le jeu de données iris, évaluez le modèle

In [95]:
X = df.copy()
y = df["species"]

In [96]:
numerical_feature = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
target = ["species"]

In [97]:
from sklearn.model_selection import train_test_split

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.preprocessing import StandardScaler

In [22]:
scaler = StandardScaler()
scaler.fit(X_train[numerical_feature])
X_train[numerical_feature] = scaler.transform(X_train[numerical_feature])
X_test[numerical_feature] = scaler.transform(X_test[numerical_feature])

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [26]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC

In [27]:
svm = SVC()
model = OneVsOneClassifier(estimator=svm)

In [28]:
model.fit(X_train, y_train)

OneVsOneClassifier(estimator=SVC())

In [31]:
y_pred = model.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score, f1_score

In [33]:
accuracy_score(y_test, y_pred)

1.0

In [35]:
f1_score(y_test, y_pred, average='weighted')

np.float64(1.0)

# Introduction aux pipelines sklearn

In [105]:
url = "https://gist.githubusercontent.com/jsz4n/b7ca11015784086788022a539935d0cf/raw/a8c3abf0a31f5c0df5e0ddd76fb9b289bac9bed1/iris.csv"

In [128]:
df = pd.read_csv(url)

In [172]:
X = df.copy().drop("species", axis=1)
y = df["species"].copy()

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [174]:
from sklearn.preprocessing import OneHotEncoder, TargetEncoder

In [175]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [176]:
categorical_feature = []

In [177]:
numerical_pipeline = Pipeline( steps=[
    ("normalization", StandardScaler())
])
categorical_pipeline = Pipeline( steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])
target_pipeline = Pipeline( steps= [
    ("label encoding", LabelEncoder())
])
preprocessing = ColumnTransformer(transformers =
        [
            # ("target_pipeline", target_pipeline, ["species"]),
            ("numerical_pipeline", numerical_pipeline, numerical_feature),
            ("categorical_pipeline", categorical_pipeline, categorical_feature)
        ]
    ,remainder="passthrough")

In [178]:
preprocessing

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical_pipeline',
                                 Pipeline(steps=[('normalization',
                                                  StandardScaler())]),
                                 ['sepal_length', 'sepal_width', 'petal_length',
                                  'petal_width']),
                                ('categorical_pipeline',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 [])])

In [179]:
full_pipeline = Pipeline( steps= [
    ("preprocessing", preprocessing),
    ("classifier", OneVsOneClassifier(estimator=SVC()))
])

In [180]:
preprocessing.set_output(transform="pandas")
preprocessing.fit_transform(X_train).head()

numerical_pipeline__sepal_length  numerical_pipeline__sepal_width  \
22                         -1.473937                         1.220379   
15                         -0.133071                         3.020017   
65                          1.085898                         0.095606   
11                         -1.230143                         0.770470   
42                         -1.717731                         0.320560   

    numerical_pipeline__petal_length  numerical_pipeline__petal_width  
22                         -1.563987                        -1.309484  
15                         -1.277280                        -1.042922  
65                          0.385621                         0.289886  
11                         -1.219939                        -1.309484  
42                         -1.391963                        -1.309484

In [181]:
y_train = le.fit_transform(y_train)

In [182]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical_pipeline',
                                                  Pipeline(steps=[('normalization',
                                                                   StandardScaler())]),
                                                  ['sepal_length',
                                                   'sepal_width',
                                                   'petal_length',
                                                   'petal_width']),
                                                 ('categorical_pipeline',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('classifier', OneVsOneClassifier(estimator=SVC()))])

In [183]:
full_pipeline.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])